In [7]:
import numpy as np
from scipy.optimize import linprog

# 目标函数的系数（求最大化时需要取负）
c = [-1, -1]

# 不等式约束的系数矩阵
A = [
    [-5/3],    # x1 - x2 <= 0
    [-5/3]    # 50x1 + 20x2 <= 2000
]

# 不等式约束的常数项
b = [0, -5.64]

# 变量的取值范围
x_bounds = (0, None)
y_bounds = (0, None)

# 调用linprog函数求解
result = linprog(c, A_ub=A, b_ub=b, bounds=[x_bounds, y_bounds], method='highs')

# 输出结果
if result.success:
    print(f"最优解：桌子数量 = {result.x[0]:.0f}, 椅子数量 = {result.x[1]:.0f}")
    print(f"最大化的总数 = {-result.fun:.0f}")
else:
    print("没有找到最优解")


ValueError: Invalid input for linprog: A_ub must have exactly two dimensions, and the number of columns in A_ub must be equal to the size of c

In [10]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

# 读取数据
path = r"C:/Users/海棠微雨/Desktop/cumcm2021c/C/附件1 近5年402家供应商的相关数据.xlsx"
df = pd.read_excel(path, sheet_name='供应商的供货量（m³）')

# 假设前50家供应商的ID已经存储在一个列表中
top_50_suppliers = suppliers[top_50_indices]  # 这里填入前50家供应商的ID

# 过滤出前50家供应商的数据
df_top_50 = df[df['供应商ID'].isin(top_50_suppliers)]

# 提取A、B、C类材料的供货量数据
data_A = df_top_50[df_top_50['材料分类'] == 'A'].iloc[:, 2:].values
data_B = df_top_50[df_top_50['材料分类'] == 'B'].iloc[:, 2:].values
data_C = df_top_50[df_top_50['材料分类'] == 'C'].iloc[:, 2:].values

# 目标函数：最小化总供应商数量
c = np.ones(data_A.shape[0] + data_B.shape[0] + data_C.shape[0])

# 约束条件：x/0.6 + y/0.66 + z/0.72 > 5.64
A_ub = np.zeros((1, c.size))
A_ub[0, :data_A.shape[0]] = 1 / 0.6
A_ub[0, data_A.shape[0]:data_A.shape[0] + data_B.shape[0]] = 1 / 0.66
A_ub[0, data_A.shape[0] + data_B.shape[0]:] = 1 / 0.72
b_ub = [-5.64 * 10000]  # 转换为立方米

# 变量的取值范围
bounds = [(0, None) for _ in range(c.size)]

# 调用linprog函数求解
result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')

# 输出结果
if result.success:
    print("最优解：")
    for i, x in enumerate(result.x):
        if x > 0:
            if i < data_A.shape[0]:
                supplier = df_top_50[df_top_50['材料分类'] == 'A'].iloc[i, 0]
            elif i < data_A.shape[0] + data_B.shape[0]:
                supplier = df_top_50[df_top_50['材料分类'] == 'B'].iloc[i - data_A.shape[0], 0]
            else:
                supplier = df_top_50[df_top_50['材料分类'] == 'C'].iloc[i - data_A.shape[0] - data_B.shape[0], 0]
            print(f"{supplier}: {x:.2f} 立方米")
    print(f"满足需求的最小供应商数量 = {np.sum(result.x > 0)}")
else:
    print("没有找到最优解")


NameError: name 'scores' is not defined